In [68]:
!pip -q install -U scikit-learn pandas numpy openpyxl xarray netCDF4 matplotlib plotly tensorflow


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from calendar import monthrange
import xarray as xr
import pandas as pd
import plotly.express as px
import plotly.io as pio
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import plotly.graph_objects as go
from typing import Literal, Optional, Dict, Any
from sklearn.preprocessing import StandardScaler, OneHotEncoder
pio.renderers.default = "notebook_connected"  # ou "vscode", "jupyterlab", etc.

## Extração dos Dados

### Download dos dados a partir do earthdata

Para utilizar devidamente esse script é necessário se cadastrar em https://urs.earthdata.nasa.gov, após isso, crie um .env com suas credenciais para fazer o download desse dataset, ajuste as variáveis iniciais de acordo com o folder desejado e range da data de download. Foi implementado paralelismo para acelerar o processo de download, ajuste conforme necessário.

In [65]:
import os
import sys
import time
import netrc
import pathlib
import platform
from datetime import datetime, timedelta
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Iterable, Tuple, List, Optional

import requests


START = "2015-01-01"     # YYYY-MM-DD
END   = "2017-12-31"     # YYYY-MM-DD (inclusivo)
OUTDIR = "./dados_gpcp"  # pasta de saída
WORKERS = 4              # paralelismo
SKIP_EXISTING = True     # pular arquivos já existentes

# -------------------- Constantes do produto --------------------
EARTHDATA_HOST = "urs.earthdata.nasa.gov"
PRODUCT_ROOT = "https://data.gesdisc.earthdata.nasa.gov/data/GPCP/GPCPDAY.3.3"
VERSION_TAG = "V3.3"  # versão do produto

# --- Loader .env (zero-deps) ---
def load_dotenv_simple(path: str = ".env") -> None:
    p = pathlib.Path(path)
    if not p.exists():
        print(f"[aviso] .env não encontrado em {p.resolve()}")
        return
    for line in p.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        k, v = line.split("=", 1)
        os.environ.setdefault(k.strip(), v.strip().strip('"').strip("'"))

load_dotenv_simple(".env")

# -------------------- Helpers --------------------
def get_credentials() -> Tuple[Optional[str], Optional[str]]:
    """Tenta obter (user, pwd) de env vars -> NETRC explícito -> _netrc/.netrc."""
    user = os.getenv("EARTHDATA_USERNAME")
    pwd  = os.getenv("EARTHDATA_PASSWORD")
    if user and pwd:
        return user, pwd

    netrc_env = os.getenv("NETRC")
    if netrc_env and pathlib.Path(netrc_env).exists():
        try:
            a = netrc.netrc(netrc_env)
            c = a.authenticators(EARTHDATA_HOST)
            if c and len(c) >= 3:
                return c[0], c[2]
        except Exception:
            pass

    home = pathlib.Path.home()
    candidates = [home / ".netrc"]
    if platform.system().lower().startswith("win"):
        candidates.insert(0, home / "_netrc")

    for p in candidates:
        if p.exists():
            try:
                a = netrc.netrc(str(p))
                c = a.authenticators(EARTHDATA_HOST)
                if c and len(c) >= 3:
                    return c[0], c[2]
            except Exception:
                continue

    return None, None

def human(n: float) -> str:
    for unit in ['','K','M','G','T','P']:
        if abs(n) < 1024.0:
            return f"{n:3.1f}{unit}B"
        n /= 1024.0
    return f"{n:.1f}EB"

def filename_from_url(url: str) -> str:
    return pathlib.Path(urlparse(url).path).name or "download.bin"

def daterange(start: datetime, end: datetime) -> Iterable[datetime]:
    cur = start
    step = timedelta(days=1)
    while cur <= end:
        yield cur
        cur += step

def build_gpcp_url(day: datetime) -> str:
    yyyy = day.strftime("%Y")
    ymd  = day.strftime("%Y%m%d")
    fname = f"GPCPDAY_L3_{ymd}_{VERSION_TAG}.nc4"
    return f"{PRODUCT_ROOT}/{yyyy}/{fname}"

def session_with_auth(user: Optional[str]=None, pwd: Optional[str]=None, token: Optional[str]=None) -> requests.Session:
    """Cria sessão; se houver token, usa Bearer; senão, usa Basic (user,pwd)."""
    s = requests.Session()
    s.headers.update({"User-Agent": "iDEV-downloader/1.3 (Notebook)"})
    if token:
        s.headers["Authorization"] = f"Bearer {token}"
    elif user and pwd:
        s.auth = (user, pwd)
    return s

def earthdata_fetch(session: requests.Session, url: str, auth_pair: Tuple[Optional[str], Optional[str]], token: Optional[str]=None, timeout=60) -> requests.Response:
    """
    Faz GET já com a auth disponível.
    Se 401 apontar para fluxo OAuth do URS, sugere consent/login no navegador.
    """
    user, pwd = auth_pair
    if token:
        r = session.get(url, stream=True, allow_redirects=True, timeout=timeout)
    elif user and pwd:
        r = session.get(url, stream=True, allow_redirects=True, timeout=timeout, auth=(user, pwd))
    else:
        r = session.get(url, stream=True, allow_redirects=True, timeout=timeout)

    if r.status_code == 401 and "urs.earthdata.nasa.gov/oauth/authorize" in r.url:
        raise RuntimeError(
            "HTTP 401 via URS/OAuth. Se estiver usando token, verifique validade/escopo. "
            "Caso seja o primeiro acesso, abra a URL do dataset no navegador, faça login e autorize o domínio "
            "(consent) para sua conta Earthdata. Depois rode novamente."
        )
    return r

def download_one(url: str, outdir: pathlib.Path, skip_existing: bool, retries=3, backoff=2.0) -> pathlib.Path:
    outdir.mkdir(parents=True, exist_ok=True)
    outpath = outdir / filename_from_url(url)

    if skip_existing and outpath.exists() and outpath.stat().st_size > 0:
        print(f"[skip] {outpath.name} ({human(outpath.stat().st_size)})")
        return outpath

    user, pwd = get_credentials()
    token = os.getenv("EARTHDATA_TOKEN")
    s = session_with_auth(user, pwd, token)

    last_err = None
    for attempt in range(1, retries + 1):
        try:
            r = earthdata_fetch(s, url, (user, pwd), token=token)
            if r.status_code == 404:
                raise FileNotFoundError(f"HTTP 404 (não encontrado): {url}")
            if r.status_code != 200:
                raise RuntimeError(f"HTTP {r.status_code} – URL final: {r.url}")

            total = int(r.headers.get("Content-Length", 0))
            downloaded = 0
            start = time.time()

            tmp_path = outpath.with_suffix(outpath.suffix + ".part")
            with open(tmp_path, "wb") as f:
                for chunk_bytes in r.iter_content(chunk_size=1024*1024):
                    if not chunk_bytes:
                        continue
                    f.write(chunk_bytes)
                    downloaded += len(chunk_bytes)
                    if total > 0:
                        pct = 100 * downloaded / total
                        sys.stdout.write(f"\rBaixando {outpath.name}: {human(downloaded)}/{human(total)} ({pct:5.1f}%)")
                    else:
                        sys.stdout.write(f"\rBaixando {outpath.name}: {human(downloaded)}")
                    sys.stdout.flush()

            tmp_path.replace(outpath)
            dur = time.time() - start
            sys.stdout.write(f"\n[ok] {outpath.name} em {dur:.1f}s ({human(downloaded)})\n")
            return outpath

        except Exception as e:
            last_err = e
            if attempt < retries:
                wait = backoff ** (attempt - 1)
                print(f"[tentativa {attempt}/{retries} falhou] {e} → retry em {wait:.1f}s")
                time.sleep(wait)
            else:
                print(f"[erro] {e}")

    raise RuntimeError(f"Falha definitiva ao baixar: {url} → {last_err}")

# -------------------- Execução --------------------
try:
    start_dt = datetime.strptime(START, "%Y-%m-%d")
    end_dt   = datetime.strptime(END,   "%Y-%m-%d")
except ValueError as e:
    raise SystemExit(f"[erro] Datas inválidas: {e}\nUse o formato YYYY-MM-DD.")

if end_dt < start_dt:
    raise SystemExit("[erro] END não pode ser anterior a START.")

urls = [build_gpcp_url(d) for d in daterange(start_dt, end_dt)]
print(f"Total de arquivos a baixar: {len(urls)}")

outdir_path = pathlib.Path(OUTDIR).resolve()
failed: List[str] = []

if WORKERS <= 1:
    for u in urls:
        try:
            download_one(u, outdir_path, skip_existing=SKIP_EXISTING)
        except Exception as e:
            failed.append(f"{u} → {e}")
else:
    with ThreadPoolExecutor(max_workers=WORKERS) as pool:
        futs = {pool.submit(download_one, u, outdir_path, SKIP_EXISTING): u for u in urls}
        for fut in as_completed(futs):
            try:
                fut.result()
            except Exception as e:
                failed.append(f"{futs[fut]} → {e}")

print("\nConcluído.")
if failed:
    print("\nFalhas:")
    for item in failed:
        print(" -", item)


Total de arquivos a baixar: 1096
[skip] GPCPDAY_L3_20150101_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150102_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150103_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150104_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150106_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150105_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150107_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150108_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150109_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150110_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150112_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150113_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150111_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150114_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150115_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150117_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150116_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150118_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150120_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150119_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150121_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150123_V3.3.nc4 (1.5MB)

### Leitura e concatenação de todos nc4 em um único dataframe

In [2]:
import re
import os
import xarray as xr
from pathlib import Path

# -------------------- Config --------------------
FOLDER = "./dados_gpcp"            # pasta com os .nc4 baixados
VARIABLE = None                    
OUTPUT_CSV = "gpcp_daily_concat.csv"
OUTPUT_PARQUET = "gpcp_daily_concat.parquet"

NORMALIZE_LON = True               # converte lon para [-180, 180]
SORT_OUTPUT = True                 # ordena por (date, lat, lon)

#  bounding box do Pará – ative se quiser filtrar já aqui
USE_PA_BBOX = False
PA_BBOX = dict(lon_min=-60.5, lon_max=-46.0, lat_min=-9.8, lat_max=2.5)

# -------------------- Funções utilitárias --------------------
DATE_RE = re.compile(r"(\d{8})")  # captura YYYYMMDD no nome do arquivo

def parse_date_from_name(path: Path) -> np.datetime64:
    m = DATE_RE.search(path.name)
    if not m:
        raise ValueError(f"Não consegui extrair YYYYMMDD do nome: {path.name}")
    ymd = m.group(1)
    return np.datetime64(f"{ymd[:4]}-{ymd[4:6]}-{ymd[6:8]}")

def load_one_dataset(p: Path) -> xr.Dataset:
    ds = xr.open_dataset(p)
    # garante dimensão temporal (muitos diários já têm 'time' com 1 passo)
    if "time" not in ds.coords and "time" not in ds.dims:
        t = parse_date_from_name(p)
        ds = ds.expand_dims(time=[t])
    # assegura que time seja datetime64 ns
    if np.issubdtype(ds["time"].dtype, np.datetime64) is False:
        ds["time"] = xr.decode_cf(ds).get("time", ds["time"])
    return ds

def pick_main_var(ds: xr.Dataset, prefer: str | None = None) -> str:
    if prefer and prefer in ds.data_vars:
        return prefer
    ignore = {"mask", "lsmask", "land_sea_mask", "lat_bnds", "lon_bnds"}
    for v in ds.data_vars:
        if v.lower() in ignore:
            continue
        if ds[v].ndim >= 2:
            return v
    # fallback: a primeira variável qualquer
    return list(ds.data_vars)[0]

# -------------------- Coleta e leitura --------------------
files = sorted(Path(FOLDER).glob("*.nc4"))
if not files:
    raise SystemExit(f"Nenhum .nc4 encontrado em {Path(FOLDER).resolve()}")

datasets = []
failed = []
for i, p in enumerate(files, 1):
    try:
        ds = load_one_dataset(p)
        datasets.append(ds)
        print(f"[{i}/{len(files)}] ok → {p.name}")
    except Exception as e:
        failed.append((p.name, str(e)))
        print(f"[{i}/{len(files)}] erro → {p.name}: {e}")

if not datasets:
    raise SystemExit("Nenhum dataset pôde ser aberto.")



[1/1096] ok → GPCPDAY_L3_20150101_V3.3.nc4
[2/1096] ok → GPCPDAY_L3_20150102_V3.3.nc4
[3/1096] ok → GPCPDAY_L3_20150103_V3.3.nc4
[4/1096] ok → GPCPDAY_L3_20150104_V3.3.nc4
[5/1096] ok → GPCPDAY_L3_20150105_V3.3.nc4
[6/1096] ok → GPCPDAY_L3_20150106_V3.3.nc4
[7/1096] ok → GPCPDAY_L3_20150107_V3.3.nc4
[8/1096] ok → GPCPDAY_L3_20150108_V3.3.nc4
[9/1096] ok → GPCPDAY_L3_20150109_V3.3.nc4
[10/1096] ok → GPCPDAY_L3_20150110_V3.3.nc4
[11/1096] ok → GPCPDAY_L3_20150111_V3.3.nc4
[12/1096] ok → GPCPDAY_L3_20150112_V3.3.nc4
[13/1096] ok → GPCPDAY_L3_20150113_V3.3.nc4
[14/1096] ok → GPCPDAY_L3_20150114_V3.3.nc4
[15/1096] ok → GPCPDAY_L3_20150115_V3.3.nc4
[16/1096] ok → GPCPDAY_L3_20150116_V3.3.nc4
[17/1096] ok → GPCPDAY_L3_20150117_V3.3.nc4
[18/1096] ok → GPCPDAY_L3_20150118_V3.3.nc4
[19/1096] ok → GPCPDAY_L3_20150119_V3.3.nc4
[20/1096] ok → GPCPDAY_L3_20150120_V3.3.nc4
[21/1096] ok → GPCPDAY_L3_20150121_V3.3.nc4
[22/1096] ok → GPCPDAY_L3_20150122_V3.3.nc4
[23/1096] ok → GPCPDAY_L3_20150123_V3.3.n

In [3]:
df_test = datasets[0].to_dataframe()
print(df_test)

                                precip  probability_liquid_phase  time_bnds
time       lat    lon     bnds                                             
2015-01-01  89.75 -179.75 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02
                  -179.25 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02
                  -178.75 0        0.0                       0.0 2015-01-01
...                                ...                       ...        ...
           -89.75  178.75 1        0.0                       0.0 2015-01-02
                   179.25 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02
                   179.75 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02

[518400 row

In [4]:
# -------------------- Concatenação temporal --------------------
# concatena ao longo de 'time' (cada arquivo diário vira 1 passo temporal)
ds_all = xr.concat(datasets, dim="time", join="outer", combine_attrs="override")

# escolhe a variável principal
var_name = pick_main_var(ds_all, prefer=VARIABLE)
print(f"Variável selecionada: {var_name}")

# renomeia para 'precip' (padrão), mantendo apenas o necessário
ds_main = ds_all[[var_name]].rename({var_name: "precip"})


# 1) ative o recorte e use o bbox do Pará (IBGE, extremos oficiais)
USE_PA_BBOX = True
PA_BBOX = dict(
    lon_min=-58.89833,  # Oeste  (−58°53′54″)
    lon_max=-46.06083,  # Leste  (−46°03′39″)
    lat_min=-9.84111,   # Sul    (−09°50′28″)
    lat_max= 2.59111    # Norte  (+02°35′28″)
)

# 2) normaliza longitude p/ [-180, 180] (mantém seu trecho)
if NORMALIZE_LON and ds_main.coords.get("lon") is not None:
    lon = ds_main["lon"]
    if lon.max() > 180:
        ds_main = ds_main.assign_coords(lon=(((lon + 180) % 360) - 180))

# 3) recorte robusto ao sentido das coordenadas
if USE_PA_BBOX and {"lon", "lat"}.issubset(ds_main.coords):
    # lon pode estar crescente (−180→+180) – normalmente está
    lon_vals = ds_main["lon"].values
    lon_slice = slice(PA_BBOX["lon_min"], PA_BBOX["lon_max"]) \
        if lon_vals[0] < lon_vals[-1] else slice(PA_BBOX["lon_max"], PA_BBOX["lon_min"])

    # lat às vezes vem decrescente (+90→−90);
    lat_vals = ds_main["lat"].values
    lat_slice = slice(PA_BBOX["lat_min"], PA_BBOX["lat_max"]) \
        if lat_vals[0] < lat_vals[-1] else slice(PA_BBOX["lat_max"], PA_BBOX["lat_min"])

    ds_main = ds_main.sel(lon=lon_slice, lat=lat_slice)


Variável selecionada: precip


In [5]:

# -------------------- DataFrame final --------------------
# Garante nomes padrão de coord
rename_coords = {}
for cand in ["latitude", "Latitude", "LAT"]:
    if "lat" not in ds_main.coords and cand in ds_main.coords:
        rename_coords[cand] = "lat"
for cand in ["longitude", "Longitude", "LON"]:
    if "lon" not in ds_main.coords and cand in ds_main.coords:
        rename_coords[cand] = "lon"
if rename_coords:
    ds_main = ds_main.rename(rename_coords)

# Para DataFrame "longo": (date, lat, lon, precip)
df = ds_main.to_dataframe().reset_index()

# Renomeia coluna temporal para 'date'
if "time" in df.columns:
    df = df.rename(columns={"time": "date"})

# Colunas em ordem amigável
cols = [c for c in ["date", "lat", "lon", "precip"] if c in df.columns] + \
       [c for c in df.columns if c not in {"date", "lat", "lon", "precip"}]
df_chuva = df[cols]

# Ordena (opcional)
if SORT_OUTPUT:
    sort_cols = [c for c in ["date", "lat", "lon"] if c in df.columns]
    if sort_cols:
        df = df.sort_values(sort_cols).reset_index(drop=True)


# -------------------- Resumo --------------------
print(f"\n✅ DataFrame criado: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
if "date" in df.columns:
    print(f"   período: {pd.to_datetime(df['date']).min().date()} → {pd.to_datetime(df['date']).max().date()}")
if failed:
    print("\n⚠️ Arquivos com erro:")
    for name, err in failed[:10]:
        print(f" - {name}: {err}")
    if len(failed) > 10:
        print(f"... e mais {len(failed)-10} erros.")


✅ DataFrame criado: 712,400 linhas x 4 colunas
   período: 2015-01-01 → 2017-12-31


### KNN para definir mesorregiões paraenses

In [6]:

USE_PA_BBOX = True
PA_BBOX = dict(lon_min=-58.89833, lon_max=-46.06083,  # Oeste/Leste
               lat_min=-9.84111,  lat_max= 2.59111)   # Sul/Norte

if USE_PA_BBOX:
    df_chuva = df_chuva[
        (df_chuva['lon'] >= PA_BBOX['lon_min']) & (df_chuva['lon'] <= PA_BBOX['lon_max']) &
        (df_chuva['lat'] >= PA_BBOX['lat_min']) & (df_chuva['lat'] <= PA_BBOX['lat_max'])
    ].copy()

# ------------- centróides aproximados das mesorregiões (WGS84) -------------

meso_centroids = pd.DataFrame([
    # name                                lat      lon
    ["Baixo Amazonas (PA)",              -1.94194,  -54.73780],  # região Santarém/Óbidos/Oriximiná
    ["Marajó (PA)",                      -1.68194, -50.48000],  # ilha do Marajó (Soure/Salvaterra)
    ["Metropolitana de Belém (PA)",      -1.45583, -48.50390],  # Belém/Ananindeua
    ["Nordeste Paraense (PA)",           -2.41889, -48.15194],  # Capanema/Bragança/Castanhal eixo
    ["Sudoeste Paraense (PA)",           -3.20278, -52.20583],  # Itaituba/Novo Progresso/Altamira S-O
    ["Sudeste Paraense (PA)",            -6.06778, -49.90194],  # Marabá/Parauapebas/Redenção
], columns=["mesoregion","lat_c","lon_c"])

# ---------------- FUNÇÕES ----------------
def haversine_np(lat1, lon1, lat2, lon2):
    """
    Distância Haversine vetorizada (km).
    """
    R = 6371.0088  # raio médio da Terra em km
    # converte para radianos
    lat1_rad = np.radians(lat1); lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2); lon2_rad = np.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def assign_mesoregion_knn(df_points, centroids):
    """
    df_points: DataFrame com colunas ['lat','lon']
    centroids: DataFrame com ['mesoregion','lat_c','lon_c']
    Retorna Series com rótulo da mesorregião (string) por linha de df_points.
    """
    # matriz distâncias: (n_points, n_centroids)
    dists = haversine_np(
        df_points['lat'].to_numpy()[:, None],
        df_points['lon'].to_numpy()[:, None],
        centroids['lat_c'].to_numpy()[None, :],
        centroids['lon_c'].to_numpy()[None, :]
    )
    idx_min = np.argmin(dists, axis=1)
    return pd.Series(centroids['mesoregion'].to_numpy()[idx_min], index=df_points.index)


### Agrupando mesoregião para agregar média de precipitação

In [7]:
coords = (
    df_chuva[['lat','lon']].drop_duplicates().reset_index(drop=True)
)

coords['mesoregion'] = assign_mesoregion_knn(coords, meso_centroids)

for col in ['mesoregion', 'mesoregion_x', 'mesoregion_y']:
    if col in df_chuva.columns:
        df_chuva = df_chuva.drop(columns=col)

# mapeia (lat, lon) -> mesoregion sem usar merge (evita conflitos)
coord_to_meso = { (r.lat, r.lon): r.mesoregion for r in coords.itertuples(index=False) }
df_chuva['mesoregion'] = list(map(coord_to_meso.get, zip(df_chuva['lat'], df_chuva['lon'])))

# --- agregação diária por mesorregião ---
df_meso_daily = (
    df_chuva
    .dropna(subset=['mesoregion'])
    .groupby(['date', 'mesoregion'], as_index=False)
    .agg(precip_mean=('precip', 'mean'))
)


### Agrupando semanas para agregar média de precipitação

Aqui também foi necessário criar um "lag" no dado para que as chuvas de semanas atrás possam se relacionar com o período de colheita.

O valor de lag_k e win_sizes foram definidos a partir de dados dessa pesquisa, que menciona um período de 6-9 meses de chuva para explicar a colheita da safra em questão (https://onlinelibrary.wiley.com/doi/10.1002/jsfa.10164?utm_source=chatgpt.com).

In [8]:
df_meso_daily['date'] = pd.to_datetime(df_meso_daily['date'])

# define semana ISO (início na segunda-feira)
week_start = df_meso_daily['date'] - pd.to_timedelta(df_meso_daily['date'].dt.weekday, unit='D')
df_meso_daily['week_start'] = week_start
df_meso_daily['week_end'] = df_meso_daily['week_start'] + pd.Timedelta(days=6)

# acumulado semanal por mesorregião (mm/semana)
df_meso_weekly = (
    df_meso_daily
    .groupby(['mesoregion', 'week_start', 'week_end'], as_index=False)
    .agg(precip_sum_week_mm=('precip_mean', 'sum'))  # 'precip_mean' está em mm/dia → soma = mm/semana
    .sort_values(['mesoregion', 'week_start'])
    .reset_index(drop=True)
)
# --- parâmetros ---
LAG_K = 36       
WIN_SIZES = [8]  

FEATURE_TAGS = [f"precip_rollsum_{w}w_end_lag_{LAG_K}w" for w in WIN_SIZES]

# se for só uma janela, pode usar um único string:
FEATURE_TAG = FEATURE_TAGS[0] if len(FEATURE_TAGS) == 1 else "__".join(FEATURE_TAGS)
# --- features defasadas (sem vazamento) ---
df_feat = (
    df_meso_weekly
    .sort_values(['mesoregion','week_start'])
    .copy()
)

# lag simples (semana exata K atrás)
df_feat[f'precip_sum_lag_{LAG_K}w'] = (
    df_feat.groupby('mesoregion')['precip_sum_week_mm'].shift(LAG_K)
)

# acumulados: soma das últimas W semanas, terminando K semanas antes
for W in WIN_SIZES:
    df_feat[f'precip_rollsum_{W}w_end_lag_{LAG_K}w'] = (
        df_feat.groupby('mesoregion')['precip_sum_week_mm']
              .apply(lambda s: s.shift(LAG_K).rolling(W, min_periods=1).sum())
              .reset_index(level=0, drop=True)
    )

# mantém só chave + features (evita usar chuva da semana corrente)
df_feat = df_feat[['mesoregion','week_start'] + [c for c in df_feat.columns if c.startswith('precip_')]]

# --- corte dinâmico para evitar NaN por falta de histórico ---
weeks_needed = LAG_K + (max(WIN_SIZES) if WIN_SIZES else 0) - 1
cutoff = df_meso_weekly['week_start'].min() + pd.Timedelta(weeks=max(0, weeks_needed))
df_meso_weekly = df_feat[df_feat['week_start'] >= cutoff].reset_index(drop=True)

cutoff = pd.Timestamp('2016-01-01') 
df_meso_weekly = (
    df_meso_weekly[df_meso_weekly['week_start'] >= cutoff]
    .reset_index(drop=True)
)



In [9]:
print(df_meso_weekly.tail())

                 mesoregion week_start  precip_sum_week_mm  \
619  Sudoeste Paraense (PA) 2017-11-27           14.480295   
620  Sudoeste Paraense (PA) 2017-12-04           18.697744   
621  Sudoeste Paraense (PA) 2017-12-11           85.864632   
622  Sudoeste Paraense (PA) 2017-12-18           38.662868   
623  Sudoeste Paraense (PA) 2017-12-25           51.629360   

     precip_sum_lag_36w  precip_rollsum_8w_end_lag_36w  
619           66.149475                     634.279564  
620           68.375801                     594.073883  
621           63.364788                     587.023556  
622           85.780838                     581.105656  
623           47.514755                     529.973698  


In [10]:
acai_dataset = pd.read_excel("./Acai_Datasets/Lavoura_Permanente_Acai_Para.xlsx",  sheet_name='Quantidade produzida (Tonela...',header=None)
acai_dataset

,0,1,2,3,4,5,6,7,8,9,10,11
0,"Tabela 1613 - Área destinada à colheita, área ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Variável - Quantidade produzida (Toneladas),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nível,Cód.,Unidade da Federação e Mesorregião Geográfica,Ano x Produto das lavouras permanentes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2016,2017,2018,2019,2020,2021,2022,2023,2024
4,NaN,NaN,NaN,Açaí,Açaí,Açaí,Açaí,Açaí,Açaí,Açaí,Açaí,Açaí
5,UF,11,Rondônia,-,1152,1858,2242,2260,1441,2115,2664,2629
6,UF,12,Acre,-,-,-,-,-,200,20,181,129
7,UF,13,Amazonas,9576,52785,62329,67757,73538,83321,90616,105211,91345
8,UF,14,Roraima,851,3513,3449,4153,4271,1957,2749,3087,3222
9,UF,15,Pará,1080612,1274056,1230699,1320150,1389941,1388116,1595455,1576302,1612142


## Tratamento dos Dados

 Necessário converter os dados relacionados a açaí para o formato Série Temporal (por semana).

In [11]:
def make_training_data_from_single(acai_dataset: pd.DataFrame, years=None) -> pd.DataFrame:
    """
    Extrai (mesoregion, ano, toneladas) do sheet 'estilo IBGE' onde os anos estão em uma linha
    e os valores anuais em colunas. Detecta automaticamente a linha de anos.
    Retorna um DF long com ['mesoregion','ano','toneladas'].
    """

    # 1) Encontrar a linha que contém mais "anos" (inteiros 1800..2100)
    def _count_years(row: pd.Series) -> int:
        nums = pd.to_numeric(row, errors="coerce")
        return int(((nums >= 1800) & (nums <= 2100)).sum())

    year_row_idx = acai_dataset.apply(_count_years, axis=1).idxmax()

    # 2) Construir mapa coluna->ano usando a linha encontrada
    year_map = {}
    for c in acai_dataset.columns:
        val = acai_dataset.iloc[year_row_idx, c]
        val_num = pd.to_numeric(val, errors="coerce")
        if pd.notna(val_num):
            yn = int(val_num)
            if 1800 <= yn <= 2100:
                year_map[c] = yn

    if not year_map:
        raise ValueError("Não encontrei uma linha com anos no acai_dataset.")

    year_cols = sorted(year_map.keys(), key=lambda k: year_map[k])

    # 3) Filtrar mesorregiões (linhas com 0 == 'ME' e nome em col 2)
    mask_me = acai_dataset[0].astype(str).eq("ME") & acai_dataset[2].notna()
    df_me = acai_dataset.loc[mask_me, [2] + year_cols].copy()

    # 4) Wide -> long
    long = df_me.melt(id_vars=[2], value_vars=year_cols,
                      var_name="col_idx", value_name="toneladas")

    # 5) Mapear coluna para ano e limpar
    long["ano"] = long["col_idx"].map(year_map).astype(int)
    long = long.drop(columns=["col_idx"]).rename(columns={2: "mesoregion"})
    long["mesoregion"] = long["mesoregion"].astype(str).str.strip()
    long["toneladas"] = pd.to_numeric(long["toneladas"], errors="coerce")

    # 6) Filtrar anos desejados (opcional)
    if years is not None:
        years = set(int(y) for y in years)
        long = long[long["ano"].isin(years)]

    # 7) Final
    return (
        long[["mesoregion", "ano", "toneladas"]]
        .dropna(subset=["toneladas"])
        .reset_index(drop=True)
    )



training_data_df = make_training_data_from_single(acai_dataset, years=[2016, 2017])
print(training_data_df.tail())

                     mesoregion   ano  toneladas
7                   Marajó (PA)  2017     324528
8   Metropolitana de Belém (PA)  2017     185650
9        Nordeste Paraense (PA)  2017     691025
10       Sudoeste Paraense (PA)  2017       4821
11        Sudeste Paraense (PA)  2017      54796


Agora os dados serão convertidos para um modelo de semana em semana, utilizando como referência os seguintes valores de distribuição de produção (em toneladas)

- Mensal (BRS Pará, sem irrigação): Jan 15, Fev 10, Mar 3, Abr 2, Mai 0, Jun 0, Jul 0, Ago 3, Set 7, Out 15, Nov 25, Dez 20. (https://www.infoteca.cnptia.embrapa.br/infoteca/bitstream/doc/1101575/1/SistemadeproducaoAcai2018.pdf)
- Semestres (Pará, sem irrigação): S1 (jan–jun) ~20–30% · S2 (jul–dez) ~70–80%.  https://www.alice.cnptia.embrapa.br/bitstream/doc/994953/1/CULTIVO20.pdf

In [12]:
MONTH_WEIGHTS = {1:0.15,2:0.10,3:0.03,4:0.02,5:0.00,6:0.00,7:0.00,8:0.03,9:0.07,10:0.15,11:0.25,12:0.20}

def mondays_in_month(year: int, month: int) -> pd.DatetimeIndex:
    start = pd.Timestamp(year=year, month=month, day=1)
    end = pd.Timestamp(year=year, month=month, day=monthrange(year, month)[1])
    d = pd.date_range(start, end, freq="W-MON")
    if start.weekday() == 0 and (len(d) == 0 or d[0] != start):
        d = d.insert(0, start)
    return d[d.month == month]

def generate_weekly_series_meso(df_meso: pd.DataFrame, seed: int = 42) -> pd.DataFrame:
    """
    df_meso: colunas ['mesoregion','ano','toneladas'] (um total anual por mesorregião)
    retorna: semanal com ['mesoregion','date','year','month','week','toneladas_semana','dist_ano','dist_mes','periodo']
    """
    rng = np.random.default_rng(seed)
    if not np.isclose(sum(MONTH_WEIGHTS[m] for m in range(1,7)), 0.30) or \
       not np.isclose(sum(MONTH_WEIGHTS[m] for m in range(7,13)), 0.70):
        raise ValueError("MONTH_WEIGHTS deve somar 30% (jan-jun) e 70% (jul-dez).")

    rows = []
    for _, r in df_meso.iterrows():
        reg  = str(r['mesoregion'])
        year = int(r['ano'])
        total_year = float(r['toneladas'])

        for m in range(1,13):
            w = MONTH_WEIGHTS[m]
            month_total = total_year * w
            weeks = mondays_in_month(year, m)
            n = len(weeks)
            if n == 0: 
                continue
            alpha = 10.0
            shares = np.zeros(n) if w == 0.0 else rng.dirichlet(np.full(n, alpha))
            tons = month_total * shares
            for d, t in zip(weeks, tons):
                rows.append({
                    'mesoregion': reg,
                    'date': d.normalize(),
                    'year': year,
                    'month': m,
                    'week': int(d.isocalendar().week),
                    'toneladas_semana': float(t),
                    'dist_ano': float(t / total_year if total_year > 0 else 0.0),
                    'dist_mes': float(t / month_total) if month_total > 0 else (0.0 if w == 0 else np.nan),
                    'periodo': 'entressafra' if m <= 6 else 'safra'
                })
    return pd.DataFrame(rows).sort_values(['mesoregion','year','date']).reset_index(drop=True)

def add_autoregressive_weekly_features(
    df_weekly: pd.DataFrame,
    group_col: str = "mesoregion",
    date_col: str = "date",
    y_col: str = "toneladas_semana",
    lags: list[int] = (1, 2, 4),
    ma_windows: list[int] = (4, ),
    keep_na: bool = True,
) -> pd.DataFrame:
    """
    Adiciona lags do alvo e médias móveis que usam SOMENTE passado.
    - lags: semanas defasadas do y (ex.: 1, 2, 4)
    - ma_windows: janelas (em semanas) p/ médias móveis; sempre com shift(1) antes do rolling para evitar vazamento.
    - keep_na: se False, remove linhas iniciais que ficaram NaN por falta de histórico.
    """
    d = df_weekly.copy()
    # garantir ordenação temporal por grupo
    d = d.sort_values([group_col, date_col])

    # lags da produção (sem vazamento)
    for k in lags:
        d[f"{y_col}_lag_{k}w"] = (
            d.groupby(group_col)[y_col].shift(k)
        )

    # médias móveis do passado (shift antes do rolling!)
    for w in ma_windows:
        d[f"{y_col}_ma_{w}w"] = (
            d.groupby(group_col)[y_col]
             .shift(1)                                 # garante só passado
             .rolling(w, min_periods=max(2, w//2))
             .mean()
        )

    if not keep_na:
        needed = max([0] + list(lags) + list(ma_windows))
        # regra simples: se quiser cortar o início por grupo quando não há histórico suficiente
        d = d.groupby(group_col, group_keys=False).apply(
            lambda g: g.iloc[needed:] if len(g) > needed else g.iloc[0:0]
        )

    return d.reset_index(drop=True)


weekly_series_meso = generate_weekly_series_meso(training_data_df, seed=123)

weekly_series_meso = add_autoregressive_weekly_features(
    weekly_series_meso,
    lags=[1, 2, 4],
    ma_windows=[4, 8],
    keep_na=True 
)
weekly_series_meso

,mesoregion,date,year,month,week,toneladas_semana,dist_ano,dist_mes,periodo,toneladas_semana_lag_1w,toneladas_semana_lag_2w,toneladas_semana_lag_4w,toneladas_semana_ma_4w,toneladas_semana_ma_8w
0,Baixo Amazonas (PA),2016-01-04,2016,1,1,243.740042,0.024785,0.165236,entressafra,NaN,NaN,NaN,NaN,NaN
1,Baixo Amazonas (PA),2016-01-11,2016,1,2,502.919283,0.051141,0.340939,entressafra,243.740042,NaN,NaN,NaN,NaN
2,Baixo Amazonas (PA),2016-01-18,2016,1,3,452.448030,0.046009,0.306724,entressafra,502.919283,243.740042,NaN,373.329662,NaN
3,Baixo Amazonas (PA),2016-01-25,2016,1,4,275.992645,0.028065,0.187101,entressafra,452.448030,502.919283,NaN,399.702452,NaN
4,Baixo Amazonas (PA),2016-02-01,2016,2,5,140.660548,0.014303,0.143035,entressafra,275.992645,452.448030,243.740042,368.775000,368.775000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,Sudoeste Paraense (PA),2017-11-27,2017,11,48,216.185104,0.044842,0.179370,safra,323.891576,317.999264,93.731387,270.699071,214.026862
620,Sudoeste Paraense (PA),2017-12-04,2017,12,49,271.945567,0.056409,0.282043,safra,216.185104,323.891576,347.174055,301.312500,222.411705
621,Sudoeste Paraense (PA),2017-12-11,2017,12,50,181.337821,0.037614,0.188071,safra,271.945567,216.185104,317.999264,282.505378,231.915665
622,Sudoeste Paraense (PA),2017-12-18,2017,12,51,229.688213,0.047643,0.238216,safra,181.337821,271.945567,323.891576,248.340017,237.896753


### Concatenação dos dados de precipitação e dos dados de produção

In [13]:

# 1) chave semanal = segunda-feira (week_start)
df_meso_weekly['week_start'] = pd.to_datetime(df_meso_weekly['week_start'])
df_meso_weekly['mesoregion'] = df_meso_weekly['mesoregion'].astype(str).str.strip()

weekly_series_meso['date'] = pd.to_datetime(weekly_series_meso['date'])
weekly_series_meso['mesoregion'] = weekly_series_meso['mesoregion'].astype(str).str.strip()
wk = weekly_series_meso.rename(columns={'date': 'week_start'}).copy()


# 2) merge limpo por mesorregião + semana (agora levando TODAS as colunas precip_ de df_meso_weekly)
precip_cols = [c for c in df_meso_weekly.columns if c.startswith('precip_')]
df_weekly_join = (
    wk.merge(
        df_meso_weekly[['mesoregion', 'week_start'] + precip_cols],
        on=['mesoregion', 'week_start'],
        how='left',
        validate='one_to_one'
    )
    .sort_values(['mesoregion', 'week_start'])
    .reset_index(drop=True)
)

df_weekly_join['week_end'] = df_weekly_join['week_start'] + pd.Timedelta(days=6)


base_cols = ['mesoregion','week_start','week_end','year','month','week',
             'toneladas_semana','periodo', 'toneladas_semana_lag_1w',	'toneladas_semana_lag_2w',	'toneladas_semana_lag_4w',
             'toneladas_semana_ma_4w',	'toneladas_semana_ma_8w']
training_data = df_weekly_join[base_cols + precip_cols]


training_data = training_data[training_data['mesoregion'] == 'Nordeste Paraense (PA)']
training_data 

,mesoregion,week_start,week_end,year,month,week,toneladas_semana,periodo,toneladas_semana_lag_1w,toneladas_semana_lag_2w,toneladas_semana_lag_4w,toneladas_semana_ma_4w,toneladas_semana_ma_8w,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_8w_end_lag_36w
312,Nordeste Paraense (PA),2016-01-04,2016-01-10,2016,1,1,27732.710719,entressafra,NaN,NaN,NaN,8605.019795,10318.222769,25.741644,66.697495,641.472336
313,Nordeste Paraense (PA),2016-01-11,2016-01-17,2016,1,2,38688.610326,entressafra,27732.710719,NaN,NaN,15419.806531,12681.623335,53.002594,65.546700,660.171753
314,Nordeste Paraense (PA),2016-01-18,2016-01-24,2016,1,3,23009.918168,entressafra,38688.610326,27732.710719,NaN,25627.305756,17395.811064,28.687334,8.286710,537.255216
315,Nordeste Paraense (PA),2016-01-25,2016-01-31,2016,1,4,15695.060788,entressafra,23009.918168,38688.610326,NaN,29810.413071,18879.019127,100.139969,35.914242,498.886483
316,Nordeste Paraense (PA),2016-02-01,2016-02-07,2016,2,5,16130.538966,entressafra,15695.060788,23009.918168,27732.710719,26281.575000,18705.908484,91.146965,21.524551,453.555245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,Nordeste Paraense (PA),2017-11-27,2017-12-03,2017,11,48,33000.812915,safra,39777.342350,36221.853064,22459.242517,40553.669901,30426.148386,12.195934,92.854034,836.052078
412,Nordeste Paraense (PA),2017-12-04,2017-12-10,2017,12,49,30536.526855,safra,33000.812915,39777.342350,63756.241670,43189.062500,31637.933094,7.283447,70.910820,831.695145
413,Nordeste Paraense (PA),2017-12-11,2017-12-17,2017,12,50,30556.999065,safra,30536.526855,33000.812915,36221.853064,34884.133796,33632.281612,56.669525,79.253830,833.102669
414,Nordeste Paraense (PA),2017-12-18,2017-12-24,2017,12,51,46866.396264,safra,30556.999065,30536.526855,39777.342350,33467.920296,34094.713539,99.665421,84.343636,762.984421


## Análise dos Dados

É possível aferir sobre a precipitação observando a média, máxima e mediana, que existe uma forte assimetria, destacando que algumas semanas são responsáveis por boa parte da precipitação do trimestre.


In [14]:
training_data.describe(include="all")

,mesoregion,week_start,week_end,year,month,week,toneladas_semana,periodo,toneladas_semana_lag_1w,toneladas_semana_lag_2w,toneladas_semana_lag_4w,toneladas_semana_ma_4w,toneladas_semana_ma_8w,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_8w_end_lag_36w
count,104,104,104,104.000000,104.000000,104.000000,104.000000,104,103.000000,102.000000,100.000000,104.000000,104.000000,104.000000,104.000000,104.000000
unique,1,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Nordeste Paraense (PA),NaN,NaN,NaN,NaN,NaN,NaN,entressafra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,104,NaN,NaN,NaN,NaN,NaN,NaN,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2016-12-29 12:00:00,2017-01-04 12:00:00,2016.500000,6.461538,26.500000,13383.336538,NaN,13219.630312,12889.760058,12536.620000,12856.978088,12341.050935,38.954868,36.872456,297.767503
min,NaN,2016-01-04 00:00:00,2016-01-10 00:00:00,2016.000000,1.000000,1.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.309547,0.089477,20.798080
25%,NaN,2016-07-02 06:00:00,2016-07-08 06:00:00,2016.000000,3.750000,13.750000,0.000000,NaN,0.000000,0.000000,0.000000,2002.815422,1919.768321,7.209070,5.797648,73.261050
50%,NaN,2016-12-29 12:00:00,2017-01-04 12:00:00,2016.500000,6.500000,26.500000,8019.639552,NaN,7705.671354,7679.071847,7300.340712,8803.588911,9459.244271,21.559301,22.062949,175.701795
75%,NaN,2017-06-27 18:00:00,2017-07-03 18:00:00,2017.000000,9.250000,39.250000,20895.109268,NaN,20567.429352,20133.422801,19684.295923,20139.759484,19863.241177,72.485437,65.834398,514.250110
max,NaN,2017-12-25 00:00:00,2017-12-31 00:00:00,2017.000000,12.000000,52.000000,63756.241670,NaN,63756.241670,63756.241670,63756.241670,43802.625000,39422.362500,154.461884,154.461884,876.992096


In [15]:
training_data.corr(numeric_only = True)

,year,month,week,toneladas_semana,toneladas_semana_lag_1w,toneladas_semana_lag_2w,toneladas_semana_lag_4w,toneladas_semana_ma_4w,toneladas_semana_ma_8w,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_8w_end_lag_36w
year,1.000000e+00,-0.005601,-1.415244e-13,-0.006571,0.017790,0.021036,0.059047,0.036792,0.089823,0.087115,0.162792,0.169944
month,-5.601384e-03,1.000000,9.964929e-01,0.428331,0.347603,0.272406,0.138332,0.251209,0.064573,-0.555362,0.597449,0.434362
week,-1.415244e-13,0.996493,1.000000e+00,0.426232,0.352591,0.280597,0.136424,0.255865,0.071275,-0.552327,0.596862,0.436019
toneladas_semana,-6.570942e-03,0.428331,4.262325e-01,1.000000,0.814561,0.786814,0.686720,0.810914,0.732704,0.079856,0.665463,0.867264
toneladas_semana_lag_1w,1.778969e-02,0.347603,3.525914e-01,0.814561,1.000000,0.815104,0.752446,0.907489,0.821533,0.165558,0.611083,0.847951
toneladas_semana_lag_2w,2.103582e-02,0.272406,2.805969e-01,0.786814,0.815104,1.000000,0.781995,0.928786,0.857480,0.289041,0.532123,0.819437
toneladas_semana_lag_4w,5.904729e-02,0.138332,1.364242e-01,0.686720,0.752446,0.781995,1.000000,0.909768,0.907344,0.414344,0.414416,0.720055
toneladas_semana_ma_4w,3.679180e-02,0.251209,2.558647e-01,0.810914,0.907489,0.928786,0.909768,1.000000,0.946554,0.332256,0.545170,0.843414
toneladas_semana_ma_8w,8.982346e-02,0.064573,7.127456e-02,0.732704,0.821533,0.857480,0.907344,0.946554,1.000000,0.531451,0.387839,0.734861
precip_sum_week_mm,8.711489e-02,-0.555362,-5.523270e-01,0.079856,0.165558,0.289041,0.414344,0.332256,0.531451,1.000000,-0.227545,0.036427


Maior quantidade de chuvas no começo e final do ano, uma menor quatidade de chuva ao meio do ano, ainda que dificilmente chegue em 0 de fato.

In [16]:
fig = px.scatter_matrix(
    training_data,
    dimensions=['precip_rollsum_8w_end_lag_36w', 'week'],
    color='mesoregion'
)
fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()

Existe uma maior produção de açaí no começo e final do ano. Meio do ano costuma apresentar menos açaí.

In [17]:
fig = px.scatter_matrix(
    training_data,
    dimensions=['toneladas_semana', 'week'],
    color='mesoregion'
)
fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()

É interessante notar que mais chuvas não significa necessariamente mais toneladas a serem produzidas, e sim que existe uma certa quantidade de chuva ideal para a produção de açaí.

Como existem muitos outliers, provavelmente o MSE será a fórmula utilizada para métrica de erros.

In [18]:

fig = px.scatter_matrix(
    training_data,
    dimensions=['precip_rollsum_8w_end_lag_36w', 'toneladas_semana'],
    color='mesoregion'
)

fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()

## Remoção dos Outliers

In [19]:
def remove_outliers_mad(
    df: pd.DataFrame,
    col: str = "toneladas_semana",
    by: list = ("mesoregion",),          # agrupa (mude se quiser)
    thresh: float = 3.5,                 # >3.5 z-MAD costuma ser outlier
    return_outliers: bool = True
):
    def _flag(g):
        x = g[col].astype(float)
        med = np.median(x)
        mad = 1.4826 * np.median(np.abs(x - med))  # MAD robust
        zmad = (x - med) / (mad + 1e-9)
        g["_is_outlier"] = np.abs(zmad) > thresh
        return g

    df2 = df.copy()
    df2 = df2.groupby(list(by), dropna=False, group_keys=False).apply(_flag)

    clean = df2.loc[~df2["_is_outlier"]].drop(columns=["_is_outlier"])
    outs  = df2.loc[df2["_is_outlier"]].drop(columns=["_is_outlier"])
    return (clean, outs) if return_outliers else clean


df, df_out = remove_outliers_mad(training_data, col="toneladas_semana", by=["mesoregion"], thresh=2.5)
df.describe(include="all")



C:\Users\Gabriel\AppData\Local\Temp\ipykernel_19520\3039602817.py:17: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,mesoregion,week_start,week_end,year,month,week,toneladas_semana,periodo,toneladas_semana_lag_1w,toneladas_semana_lag_2w,toneladas_semana_lag_4w,toneladas_semana_ma_4w,toneladas_semana_ma_8w,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_8w_end_lag_36w
count,95,95,95,95.000000,95.000000,95.000000,95.000000,95,94.000000,94.000000,92.000000,95.000000,95.000000,95.000000,95.000000,95.000000
unique,1,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Nordeste Paraense (PA),NaN,NaN,NaN,NaN,NaN,NaN,entressafra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,95,NaN,NaN,NaN,NaN,NaN,NaN,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2016-12-24 09:05:41.052631552,2016-12-30 09:05:41.052631552,2016.515789,6.105263,24.947368,10327.457356,NaN,11401.117717,11025.757961,11246.497821,11247.353995,11052.361577,38.763100,32.760807,260.972906
min,NaN,2016-01-04 00:00:00,2016-01-10 00:00:00,2016.000000,1.000000,1.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.309547,0.089477,20.798080
25%,NaN,2016-06-23 12:00:00,2016-06-29 12:00:00,2016.000000,3.000000,13.000000,0.000000,NaN,0.000000,0.000000,0.000000,1274.175295,1596.445522,7.064867,5.495067,64.881052
50%,NaN,2017-01-09 00:00:00,2017-01-15 00:00:00,2017.000000,6.000000,25.000000,5610.817307,NaN,5289.236013,5289.236013,4946.577981,7595.778965,7696.937266,19.676895,17.085066,139.484810
75%,NaN,2017-06-22 12:00:00,2017-06-28 12:00:00,2017.000000,9.000000,36.500000,16963.701742,NaN,16666.152745,17076.232427,16851.171056,17200.888642,17999.278144,73.412876,53.058357,420.113293
max,NaN,2017-12-25 00:00:00,2017-12-31 00:00:00,2017.000000,12.000000,52.000000,36221.853064,NaN,63756.241670,59573.106642,63756.241670,43802.625000,39422.362500,154.461884,154.461884,876.992096


In [20]:
fig = px.scatter_matrix(
    df,
    dimensions=['precip_rollsum_8w_end_lag_36w', 'toneladas_semana'],
    color='periodo'
)

fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()

## Desenvolvimento da Machine Learning

A técnica a ser utilizada inicialmente será uma regressão linear

### Regressores

In [72]:
class AcaiRegressor:
    def __init__(
        self,
        technique: Literal["linear", "poly2"] = "linear",
        feature_mode: Literal["engineered", "explicit"] = "engineered",
        num_cols: Optional[list] = None,     # colunas numéricas do df
        cat_cols: Optional[list] = None,     # colunas categóricas do df (one-hot)
        cyclic_cols: Optional[dict] = None,  # ex.: {"week": 52, "month": 12}
        # splits
        val_frac: float = 0.2,
        test_frac: float = 0.2,
        # model & train hparams
        optimizer: str = "adam",
        learning_rate: float = 1e-3,
        l2: float = 0.0,                 # L2 na camada linear
        epochs: int = 300,
        batch_size: Optional[int] = 32,
        early_stopping_patience: int = 80,
        early_min_delta: float = 1e-8,
        # feature flags
        use_region_ohe: bool = True,
        include_harmonic_k2: bool = True,
        include_month_cycle: bool = True,
        include_interactions: bool = True,  # precip × seno/cosseno
        loss_fn: Literal["mse", "mae"] = "mse",
        feature_spec: Optional[dict] = None,
        passthrough_cols: Optional[list] = None,

    ):


        self.technique = technique
        self.val_frac = val_frac
        self.test_frac = test_frac
        self.optimizer_name = optimizer.lower()
        self.learning_rate = learning_rate
        self.l2 = l2
        self.epochs = epochs
        self.batch_size = batch_size
        self.early_patience = early_stopping_patience
        self.early_min_delta = early_min_delta
        self.use_region_ohe = use_region_ohe
        self.include_harmonic_k2 = include_harmonic_k2
        self.include_month_cycle = include_month_cycle
        self.include_interactions = include_interactions

        # serão definidos após fit()
        self.ohe_: Optional[OneHotEncoder] = None
        self.xsc_: Optional[StandardScaler] = None
        self.ysc_: Optional[StandardScaler] = None
        self.model_: Optional[keras.Model] = None
        self.history_ = None
        self.feature_names_: Optional[list] = None
        self.splits_: Dict[str, Any] = {}

        self.feature_mode = feature_mode
        self.num_cols = num_cols or []
        self.cat_cols = cat_cols or []
        self.cyclic_cols = cyclic_cols or {}   # nome->período (int)
        self.loss_fn = loss_fn.lower()


    # ---------- feature engineering ----------
    def _build_numeric_features(self, d: pd.DataFrame) -> np.ndarray:
        names, cols = [], []
    
        if self.feature_mode == "explicit":
            # (a) passa colunas numéricas cruas
            for c in self.num_cols:
                if c in d.columns:
                    v = pd.to_numeric(d[c], errors="coerce").to_numpy()
                    cols += [v]; names += [c]
            # (b) expansões cíclicas opcionais (sin/cos) sobre colunas listadas
            for c, period in self.cyclic_cols.items():
                if c in d.columns:
                    v = pd.to_numeric(d[c], errors="coerce").to_numpy().astype(float)
                    s = np.sin(2*np.pi*v/float(period))
                    c_ = np.cos(2*np.pi*v/float(period))
                    cols += [s, c_]; names += [f"{c}_sin", f"{c}_cos"]
            X_num = np.column_stack(cols) if cols else np.empty((len(d), 0))
            self.feature_names_ = names
            return X_num
    
        # === modo "engineered" (seu comportamento anterior) ===
        # básicos
        week = d["week"].astype(float).to_numpy()
        denom_w = 52.0
        sin_w = np.sin(2*np.pi*week/denom_w)
        cos_w = np.cos(2*np.pi*week/denom_w)
        p = d["precip_rollsum_8w_end_lag_36w"].astype(float).to_numpy()
        #p = np.log1p(p)
    
        cols += [p, sin_w, cos_w]; names += ["p", "sin_w", "cos_w"]
    
        if self.include_month_cycle:
            month = d["month"].astype(float).to_numpy()
            sin_m = np.sin(2*np.pi*month/12.0)
            cos_m = np.cos(2*np.pi*month/12.0)
            cols += [sin_m, cos_m]; names += ["sin_m", "cos_m"]
    
        if self.technique == "poly2":
            cols += [p**2]; names += ["p2"]
            if self.include_interactions:
                cols += [p*sin_w, p*cos_w]; names += ["p_sin_w", "p_cos_w"]
                if self.include_month_cycle:
                    cols += [p*sin_m, p*cos_m]; names += ["p_sin_m", "p_cos_m"]
            if self.include_harmonic_k2:
                sin2w = np.sin(2*np.pi*2*week/denom_w)
                cos2w = np.cos(2*np.pi*2*week/denom_w)
                cols += [sin2w, cos2w]; names += ["sin2w", "cos2w"]
    
        X_num = np.column_stack(cols)
        self.feature_names_ = names
        return X_num


    def _build_X(self, df_train: pd.DataFrame, df_other: pd.DataFrame):
        """
        Monta X_train e X_other:
          - numéricas: vindas do _build_numeric_features(...)
          - categóricas: OHE genérico (todas as colunas informadas) com handle_unknown='ignore'
          - escala: StandardScaler fitado em X_train
        Também popula self.feature_names_full_ (numéricas + nomes OHE).
        """
        # --- 1) Numéricas ---
        Xtr_num = self._build_numeric_features(df_train)   # define self.feature_names_ (numéricas)
        Xot_num = self._build_numeric_features(df_other)
    
        # --- 2) Quais colunas categóricas serão usadas ---
        if self.feature_mode == "explicit":
            cat_cols = [c for c in (self.cat_cols or []) if c in df_train.columns]
        else:  # engineered
            cat_cols = ["mesoregion"] if (getattr(self, "use_region_ohe", True) and "mesoregion" in df_train.columns) else []
    
        # --- 3) One-Hot (genérico, para N colunas categóricas) ---
        ohe_feature_names = []
        if len(cat_cols) > 0:
            # compatibilidade: sklearn<1.2 usa 'sparse', >=1.2 tem 'sparse_output'
            try:
                self.ohe_ = OneHotEncoder(handle_unknown="ignore", sparse_output=False, dtype=float)
            except TypeError:
                self.ohe_ = OneHotEncoder(handle_unknown="ignore", sparse=False, dtype=float)
    
            Ztr = self.ohe_.fit_transform(df_train[cat_cols])
            Zot = self.ohe_.transform(df_other[cat_cols])
    
            # nomes das dummies: <col>=<categoria>
            for col, cats in zip(cat_cols, self.ohe_.categories_):
                ohe_feature_names += [f"{col}={cat}" for cat in cats]
    
        else:
            # sem categóricas -> matrizes vazias com número de linhas correto
            Ztr = np.empty((len(df_train), 0), dtype=float)
            Zot = np.empty((len(df_other), 0), dtype=float)
    
        # --- 4) Concatena numéricas + OHE ---
        Xtr = np.hstack([Xtr_num, Ztr])
        Xot = np.hstack([Xot_num, Zot])
    
        # guarda nomes completos (para debug/feature importance fora de redes etc.)
        num_names = list(self.feature_names_ or [])
        self.feature_names_full_ = num_names + ohe_feature_names
    
        # --- 5) Escala (fit só no treino) ---
        self.xsc_ = StandardScaler().fit(Xtr)
        return self.xsc_.transform(Xtr), self.xsc_.transform(Xot)
    
        


    # ---------- split temporal ----------
    def _temporal_split(self, df: pd.DataFrame):
        n = len(df)
        cut_test = int(n * (1 - self.test_frac))
        cut_val  = int(cut_test * (1 - self.val_frac))
        df_train = df.iloc[:cut_val].copy()
        df_val   = df.iloc[cut_val:cut_test].copy()
        df_test  = df.iloc[cut_test:].copy()
    
        # y
        y_train = df_train["toneladas_semana"].astype(float).to_numpy()
        y_val   = df_val["toneladas_semana"].astype(float).to_numpy()
        y_test  = df_test["toneladas_semana"].astype(float).to_numpy()
    
        # X (numéricas + OHE, escaladas) para train/val
        X_train, X_val = self._build_X(df_train, df_val)
    
        # ---------- TESTE: mesma lógica de OHE usada em _build_X/predict ----------
        X_test_num = self._build_numeric_features(df_test)
    
        use_ohe = False
        cols_test = None
        if self.feature_mode == "explicit":
            if len(self.cat_cols) > 0:
                use_ohe = True
                cols_test = self.cat_cols
        else:  # engineered
            if getattr(self, "use_region_ohe", True):
                use_ohe = True
                cols_test = ["mesoregion"]
    
        if use_ohe:
            # self.ohe_ já foi fitado em _build_X (no train)
            Ztest = self.ohe_.transform(df_test[cols_test])
            X_test = np.hstack([X_test_num, Ztest])
        else:
            X_test = X_test_num
    
        # escala de X (self.xsc_ já foi fitado em _build_X com X_train)
        X_test = self.xsc_.transform(X_test)
    
        # scaler de y (fit só no treino)
        self.ysc_ = StandardScaler().fit(y_train.reshape(-1,1))
        y_train_s = self.ysc_.transform(y_train.reshape(-1,1)).ravel()
        y_val_s   = self.ysc_.transform(y_val.reshape(-1,1)).ravel()
    
        self.splits_ = dict(
            df_train=df_train, df_val=df_val, df_test=df_test,
            X_train=X_train, y_train=y_train, y_train_s=y_train_s,
            X_val=X_val, y_val=y_val, y_val_s=y_val_s,
            X_test=X_test, y_test=y_test
        )
    

    def _build_model(self, n_features: int) -> keras.Model:
        reg = regularizers.l2(self.l2) if self.l2 > 0 else None
        inputs = keras.Input(shape=(n_features,))
        outputs = layers.Dense(1, activation="linear", use_bias=True,
                               kernel_regularizer=reg, name="linreg")(inputs)
        model = keras.Model(inputs, outputs)
    
        if self.optimizer_name == "adam":
            opt = keras.optimizers.Adam(self.learning_rate)
        elif self.optimizer_name == "sgd":
            opt = keras.optimizers.SGD(self.learning_rate, momentum=0.9, nesterov=True)
        else:
            raise ValueError("optimizer deve ser 'adam' ou 'sgd'")
    
        # NOVO: escolha da loss e métricas adicionais para monitorar
        loss_name = "mse" if self.loss_fn == "mse" else "mae"
        model.compile(
            optimizer=opt,
            loss=loss_name,
            metrics=[
                keras.metrics.MeanAbsoluteError(name="mae"),
                keras.metrics.MeanSquaredError(name="mse"),
            ],
        )
        return model


        # ---------- API pública ----------
    def fit(self, df: pd.DataFrame):
        """
        df precisa conter: 'week_start', 'toneladas_semana' e as colunas listadas
        em num_cols, cat_cols e cyclic_cols (apenas essas serão usadas).
        """
        # monta o conjunto mínimo de colunas obrigatórias
        req_cols = set(self.num_cols) | set(self.cat_cols) | set(self.cyclic_cols.keys())
        req_cols |= {"toneladas_semana", "week_start"}  # alvo + ordem temporal
    
        # checagem simples
        missing = [c for c in req_cols if c not in df.columns]
        if missing:
            raise ValueError(f"Colunas ausentes no df para o modo 'explicit': {missing}")
    
        # filtra o DF e remove NaNs só nas colunas necessárias
        df = df.loc[:, list(req_cols)].dropna(subset=list(req_cols)).sort_values("week_start").copy()
    
        self._temporal_split(df)
        Xtr, ytr_s = self.splits_["X_train"], self.splits_["y_train_s"]
        Xval, yval_s = self.splits_["X_val"], self.splits_["y_val_s"]
    
        if self.batch_size is None:
            self.batch_size = min(32, max(8, int(len(Xtr)/10)))
    
        self.model_ = self._build_model(Xtr.shape[1])
    
        es = keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=self.early_patience, min_delta=self.early_min_delta,
            restore_best_weights=True, verbose=1
        )
    
        print(f"\n[DEBUG] n_features: {Xtr.shape[1]}")
        print("[DEBUG] features numéricas:", self.feature_names_)
        if hasattr(self, "ohe_") and self.ohe_ is not None:
            print("[DEBUG] categorias OHE:", [list(c) for c in self.ohe_.categories_])
    
        self.history_ = self.model_.fit(
            Xtr, ytr_s,
            validation_data=(Xval, yval_s),
            epochs=self.epochs,
            batch_size=self.batch_size,
            shuffle=False,
            callbacks=[es],
            verbose=1
        )
        return self


    # --- troque o método predict inteiro por este ---
    def predict(self, df_new: pd.DataFrame) -> np.ndarray:
        X_num = self._build_numeric_features(df_new)
    
        # mesma decisão de OHE usada no _build_X
        use_ohe = False
        cols_infer = None
        if self.feature_mode == "explicit":
            if len(self.cat_cols) > 0:
                use_ohe = True
                cols_infer = self.cat_cols
        else:
            if getattr(self, "use_region_ohe", True):
                use_ohe = True
                cols_infer = ["mesoregion"]
    
        if use_ohe:
            Z = self.ohe_.transform(df_new[cols_infer])
            X = np.hstack([X_num, Z])
        else:
            X = X_num
    
        Xs = self.xsc_.transform(X)
        y_s = self.model_.predict(Xs, verbose=0).ravel()
        return self.ysc_.inverse_transform(y_s.reshape(-1,1)).ravel()


    def _metrics(self, y_true, y_hat) -> Dict[str, float]:
        mse  = mean_squared_error(y_true, y_hat)
        rmse = np.sqrt(mse)
        mae  = mean_absolute_error(y_true, y_hat)
        smape = float(np.mean(
            2.0 * np.abs(y_hat - y_true) / (np.abs(y_true) + np.abs(y_hat) + 1e-9)
        ))
        r2   = r2_score(y_true, y_hat)
        nrmse_std = rmse / (np.std(y_true) + 1e-9)
        nrmse_rng = rmse / (np.max(y_true) - np.min(y_true) + 1e-9)
        return dict(MSE=mse, RMSE=rmse, MAE=mae, SMAPE=smape, R2=r2,
                    NRMSE_std=nrmse_std, NRMSE_range=nrmse_rng)

    def evaluate(self) -> Dict[str, Dict[str, float]]:
        """Retorna métricas para TREINO, VALIDAÇÃO e TESTE (em toneladas)."""
        Xtr = self.splits_["X_train"]; ytr_true = self.splits_["y_train"]
        Xval = self.splits_["X_val"];  yval_true = self.splits_["y_val"]
        Xte = self.splits_["X_test"];  yte_true = self.splits_["y_test"]
    
        # preds -> toneladas
        ytr_pred = self.ysc_.inverse_transform(self.model_.predict(Xtr, verbose=0).ravel().reshape(-1,1)).ravel()
        yval_pred = self.ysc_.inverse_transform(self.model_.predict(Xval, verbose=0).ravel().reshape(-1,1)).ravel()
        yte_pred  = self.ysc_.inverse_transform(self.model_.predict(Xte,  verbose=0).ravel().reshape(-1,1)).ravel()
    
        # acurácia dentro de ±10%
        def acc_within(y_true, y_hat, tol=0.1):
            rel = np.abs(y_hat - y_true) / (np.abs(y_true) + 1e-9)
            return float((rel <= tol).mean())
    
        return dict(
            TREINO={**self._metrics(ytr_true, ytr_pred), "Acc@10%": acc_within(ytr_true, ytr_pred)},
            VALIDACAO={**self._metrics(yval_true, yval_pred), "Acc@10%": acc_within(yval_true, yval_pred)},
            TESTE={**self._metrics(yte_true, yte_pred), "Acc@10%": acc_within(yte_true, yte_pred), "Acc@20%": acc_within(yte_true, yte_pred, tol=0.2),
                    "Acc@30%": acc_within(yte_true, yte_pred, tol=0.3)},
        )

            # ======= helpers internos =======
    def _get_split_df_(self, split:str):
        key = dict(TREINO="df_train", VALIDACAO="df_val", TESTE="df_test")[split.upper()]
        return self.splits_[key].copy()
    
    def _split_arrays_(self, split:str):
        s = self.splits_
        if split.upper()=="TREINO":
            X, y = s["X_train"], s["y_train"]
        elif split.upper()=="VALIDACAO":
            X, y = s["X_val"], s["y_val"]
        else:
            X, y = s["X_test"], s["y_test"]
        # previsão em escala original (toneladas)
        y_hat_s = self.model_.predict(X, verbose=0).ravel()
        y_hat   = self.ysc_.inverse_transform(y_hat_s.reshape(-1,1)).ravel()
        return y, y_hat
    
    def _mode_or_first_(self, series: pd.Series):
        try:
            return series.mode(dropna=True).iloc[0]
        except Exception:
            return series.dropna().iloc[0] if series.dropna().shape[0] else None

    def plot_pairs_real_pred(
        self,
        split: str = "TESTE",            # "TREINO" | "VALIDACAO" | "TESTE" | "FULL"
        x_col: str = "week_start",
        tol: float | None = None, 
        series_mode: str = "markers",    # "markers" | "lines" | "lines+markers"
        show_pair_segments: bool = True,  # liga/desliga os segmentos verticais real->previsto
        spline_smooth: float = 0.6,
        tol_abs: float | None = None,    
        show_abs_band: bool = True      
    ):
    
        # --- 0) pega df do split OU FULL ---
        split_up = split.upper()
        if split_up == "FULL":
            if not self.splits_:
                raise RuntimeError("Modelo não treinado. Chame .fit(...) antes.")
            df_s = pd.concat(
                [self.splits_["df_train"], self.splits_["df_val"], self.splits_["df_test"]],
                axis=0, ignore_index=True
            ).copy()
        else:
            df_s = self._get_split_df_(split_up).copy()
    
        # --- 1) sanear mínimos obrigatórios ---
        if "toneladas_semana" not in df_s.columns or "week_start" not in df_s.columns:
            raise ValueError("Faltam colunas obrigatórias: 'toneladas_semana' e/ou 'week_start'.")
    
        # garantir datetime e ordenar por tempo para estabilidade visual
        df_s["week_start"] = pd.to_datetime(df_s["week_start"])
        # derive week/month aqui se faltarem (evita KeyError no predict em modo engineered)
        if "week" not in df_s.columns:
            df_s["week"] = df_s["week_start"].dt.isocalendar().week.astype(int)
        if getattr(self, "include_month_cycle", False) and ("month" not in df_s.columns):
            df_s["month"] = df_s["week_start"].dt.month.astype(int)
    
        # --- 2) eixo X ---
        if x_col not in df_s.columns:
            raise ValueError(f"x_col '{x_col}' não está no df do split '{split_up}'.")
        df_s = df_s.sort_values(x_col)
        if x_col == "week_start":
            x_vals = pd.to_datetime(df_s[x_col])
            x_title = "tempo (week_start)"
            x_as_num = pd.to_numeric(x_vals)  # p/ máscara de finitos
        else:
            x_vals = pd.to_numeric(df_s[x_col], errors="coerce")
            x_title = x_col
            x_as_num = x_vals
    
        # --- 3) y real e previsto (usa o MESMO pipeline do modelo já treinado) ---
        y_true = pd.to_numeric(df_s["toneladas_semana"], errors="coerce").to_numpy(dtype=float)
        y_pred = self.predict(df_s).astype(float)
    
        # --- 4) máscara de sanidade e reindexação consistente ---
        mask = np.isfinite(y_true) & np.isfinite(y_pred) & np.isfinite(x_as_num.to_numpy(dtype=float))
        df_s = df_s.loc[mask].copy()
        x_vals = x_vals[mask]
        y_true = y_true[mask]
        y_pred = y_pred[mask]
    
        # --- 5) métricas ponto a ponto ---
        err     = y_pred - y_true
        abs_err = np.abs(err)
        ape     = abs_err / (np.abs(y_true) + 1e-9)
        
        acc_pct = None
        if tol is not None:
            acc_pct = float((ape <= float(tol)).mean()) * 100.0
        
        acc_abs = None
        if tol_abs is not None:
            acc_abs = float((abs_err <= float(tol_abs)).mean()) * 100.0
        # --- 6) plot ---
        palette = [
            "#636EFA","#EF553B","#00CC96","#AB63FA","#FFA15A",
            "#19D3F3","#FF6692","#B6E880","#FF97FF","#FECB52"
        ]
        colors = [palette[i % len(palette)] for i in range(len(df_s))]
    
        # sanity dos modos
        if series_mode not in {"markers", "lines", "lines+markers"}:
            raise ValueError("series_mode deve ser 'markers', 'lines' ou 'lines+markers'.")
    
        fig = go.Figure()
    
        # segmentos conectando real -> previsto (um por amostra)
        if show_pair_segments:
            for i in range(len(df_s)):
                fig.add_trace(go.Scatter(
                    x=[x_vals.iloc[i], x_vals.iloc[i]] if hasattr(x_vals, "iloc") else [x_vals[i], x_vals[i]],
                    y=[y_true[i], y_pred[i]],
                    mode="lines",
                    line=dict(color=colors[i], width=2),
                    hoverinfo="skip",
                    showlegend=False
                ))
    
        # série REAL
        fig.add_trace(go.Scatter(
            x=x_vals, y=y_true, mode=series_mode,
            marker=dict(symbol="circle", size=9, line=dict(width=0)) if "markers" in series_mode else None,
            line=(dict(width=2, shape="spline", smoothing=spline_smooth) if "lines" in series_mode else None),
            name="Real",
            hovertemplate="%{x}<br>real %{y:.0f}<extra></extra>"
        ))
        # série PREVISTA
        fig.add_trace(go.Scatter(
            x=x_vals, y=y_pred, mode=series_mode,
            marker=dict(symbol="square", size=10, line=dict(width=2), color="rgba(0,0,0,0)") if "markers" in series_mode else None,
            line=(dict(width=2, shape="spline", smoothing=spline_smooth) if "lines" in series_mode else None),
            name="Previsto",
            hovertemplate=(
                "%{x}<br>previsto %{y:.0f}"
                "<br>|err|/real=%{customdata[0]:.1%}"
                "<br>|err|=%{customdata[1]:,.0f}<extra></extra>"
            ),
            customdata=np.c_[ape, abs_err]   # ⬅️ [% relativo, erro absoluto]
        ))

    
        # banda ±tol do real
        xv = x_vals if hasattr(x_vals, "to_numpy") else np.array(x_vals)
        
        # faixa percentual ±tol (opcional)
        if tol is not None:
            fig.add_trace(go.Scatter(
                x=np.concatenate([xv, xv[::-1]]),
                y=np.concatenate([(1+tol)*y_true, ((1-tol)*y_true)[::-1]]),
                fill="toself", fillcolor="rgba(0,150,0,0.06)",
                line=dict(color="rgba(0,0,0,0)"),
                hoverinfo="skip", showlegend=True,
                name=f"faixa ±{int(tol*100)}% do real"
            ))
        
        # faixa absoluta ±tol_abs (opcional)
        if (tol_abs is not None) and show_abs_band:
            fig.add_trace(go.Scatter(
                x=np.concatenate([xv, xv[::-1]]),
                y=np.concatenate([y_true + tol_abs, (y_true - tol_abs)[::-1]]),
                fill="toself", fillcolor="rgba(255,165,0,0.12)",
                line=dict(color="rgba(0,0,0,0)"),
                hoverinfo="skip", showlegend=True,
                name=f"faixa ±{int(tol_abs):,} abs"
            ))

        title_parts = [f"Real vs Previsto — {split_up}"]
        if acc_pct is not None:
            title_parts.append(f"Acc@{int(tol*100)}% = {acc_pct:.1f}%")
        if acc_abs is not None:
            title_parts.append(f"Acc@±{int(tol_abs):,} = {acc_abs:.1f}%")
        
        fig.update_layout(
            title="  |  ".join(title_parts),
            xaxis_title=x_title,
            yaxis_title="toneladas",
            template="plotly_white",
            hovermode="closest"
        )

        fig.show()
    

    def plot_learning_curve(self):
        loss = self.history_.history.get("loss", [])
        val_loss = self.history_.history.get("val_loss", [])
        epochs = np.arange(1, len(loss)+1)
    
        # NOVO: rótulos conforme a loss escolhida
        lname = "MSE" if self.loss_fn == "mse" else "MAE"
    
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=epochs, y=loss, mode="lines+markers",
            name=f"train {lname}(z)",
            hovertemplate="epoch %{x}<br>"+f"{lname}(z) "+"%{y:.3f}<extra></extra>"
        ))
        if len(val_loss):
            fig.add_trace(go.Scatter(
                x=epochs, y=val_loss, mode="lines+markers",
                name=f"val {lname}(z)",
                hovertemplate="epoch %{x}<br>"+f"val {lname}(z) "+"%{y:.3f}<extra></extra>"
            ))
            best = int(np.argmin(val_loss))
            fig.add_vline(x=epochs[best], line_width=1, line_dash="dash")
            fig.add_annotation(x=epochs[best], y=val_loss[best],
                               text=f"best val @{epochs[best]}: {val_loss[best]:.3f}",
                               showarrow=True, ax=20, ay=-40)
    
        fig.update_layout(
            title=f"Learning Curve — {self.technique.upper()}",
            xaxis_title="epoch",
            yaxis_title=f"{lname} (z-score y)",
            template="plotly_white"
        )
        fig.show()


### Execução dos regressores

In [73]:
print(df.columns)

Index(['mesoregion', 'week_start', 'week_end', 'year', 'month', 'week',
       'toneladas_semana', 'periodo', 'toneladas_semana_lag_1w',
       'toneladas_semana_lag_2w', 'toneladas_semana_lag_4w',
       'toneladas_semana_ma_4w', 'toneladas_semana_ma_8w',
       'precip_sum_week_mm', 'precip_sum_lag_36w',
       'precip_rollsum_8w_end_lag_36w'],
      dtype='object')


In [74]:
reg = AcaiRegressor(
    learning_rate = 1e-2,
    epochs = 500,
    batch_size= 16,
    early_stopping_patience = 80,
    val_frac = 0.2,
    test_frac = 0.3,
    feature_mode="explicit",
    technique="linear",
    num_cols=["precip_rollsum_8w_end_lag_36w", "toneladas_semana_lag_1w", "week"],
    cat_cols=['periodo'], 
    loss_fn="mse",      
    cyclic_cols={"week":df['mesoregion'].count()},
    include_month_cycle=False,
    include_harmonic_k2=True,
    include_interactions=False,
)
reg.fit(df)
print(reg.feature_names_)


[DEBUG] n_features: 7
[DEBUG] features numéricas: ['precip_rollsum_8w_end_lag_36w', 'toneladas_semana_lag_1w', 'week', 'week_sin', 'week_cos']
[DEBUG] categorias OHE: [['entressafra', 'safra']]
Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 2.6681 - mae: 1.2494 - mse: 2.6681 - val_loss: 1.3033 - val_mae: 1.0714 - val_mse: 1.3033
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2.2932 - mae: 1.1512 - mse: 2.2932 - val_loss: 1.0711 - val_mae: 0.9709 - val_mse: 1.0711
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.9845 - mae: 1.0675 - mse: 1.9845 - val_loss: 0.8737 - val_mae: 0.8753 - val_mse: 0.8737
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.7130 - mae: 0.9885 - mse: 1.7130 - val_loss: 0.7068 - val_mae: 0.7868 - val_mse: 0.7068
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.4755 - mae: 0.9127 - mse: 1.4755 - val_loss: 0.5680 - val_mae: 0.7067 - val_mse: 0.5680
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.2699 

In [75]:
metrics = reg.evaluate()
print("Teste: ", metrics["TESTE"])
print("Validação: ", metrics["VALIDACAO"])
print("Treino: ", metrics["TREINO"])

Teste:  {'MSE': 23457896.63561483, 'RMSE': np.float64(4843.335280115845), 'MAE': 3187.938578843487, 'SMAPE': 0.9170303143167392, 'R2': 0.8432934285006844, 'NRMSE_std': np.float64(0.39586180858891906), 'NRMSE_range': np.float64(0.13371307292015822), 'Acc@10%': 0.2413793103448276, 'Acc@20%': 0.27586206896551724, 'Acc@30%': 0.5172413793103449}
Validação:  {'MSE': 8252254.962821989, 'RMSE': np.float64(2872.673835091967), 'MAE': 2199.455344848715, 'SMAPE': 0.7973139049886879, 'R2': 0.7527138571006169, 'NRMSE_std': np.float64(0.49727873763041336), 'NRMSE_range': np.float64(0.13763278190699976), 'Acc@10%': 0.07692307692307693}
Treino:  {'MSE': 9188154.694330184, 'RMSE': np.float64(3031.196907878171), 'MAE': 2392.6006187016906, 'SMAPE': 0.6959431923351331, 'R2': 0.9052228320259428, 'NRMSE_std': np.float64(0.3078590066475853), 'NRMSE_range': np.float64(0.09400345921284821), 'Acc@10%': 0.15384615384615385}


In [79]:
reg.plot_pairs_real_pred(split="TESTE", x_col="week", tol=0.10)
reg.plot_pairs_real_pred(split="TESTE", x_col="week", tol_abs=400)
reg.plot_pairs_real_pred(split="FULL", x_col="week", tol=0.4, series_mode="lines", spline_smooth=0)
reg.plot_pairs_real_pred(split="FULL", x_col="week", series_mode="lines",
                          tol_abs=400)

In [71]:
reg.plot_learning_curve()       

In [487]:
df_future = df.tail(12).copy() 
y_future_hat = reg.predict(df_future)

print(y_future_hat)

[15215.009 17410.646 19781.307 21668.76  22744.291 24567.596 26886.365
 29744.94  31602.37  32659.682 33736.887 33661.65 ]


## Random Forest